# YOLOv11 - Detección de Autos en Video

In [11]:
import sys
import platform

# ============================================
# CELDA 1: Verificar sistema macOS
# ============================================
print("🍎 Verificando sistema macOS...")
print(f"Sistema: {platform.system()}")
print(f"Versión: {platform.mac_ver()[0]}")
print(f"Procesador: {platform.processor()}")
print(f"Arquitectura: {platform.machine()}")

# Detectar si es Apple Silicon
is_apple_silicon = platform.machine() == 'arm64'
if is_apple_silicon:
    print("✅ Apple Silicon detectado (M1/M2/M3)")
else:
    print("✅ Intel Mac detectado")

🍎 Verificando sistema macOS...
Sistema: Darwin
Versión: 26.0.1
Procesador: arm
Arquitectura: arm64
✅ Apple Silicon detectado (M1/M2/M3)


In [ ]:
# ============================================
# CELDA 2: Instalar dependencias
# ============================================
%pip install ultralytics opencv-python torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.


In [15]:
# ============================================
# CELDA 3: Importar librerías
# ============================================
from ultralytics import YOLO
import cv2
import numpy as np
from pathlib import Path
import torch
import os

print("\n✅ Librerías importadas correctamente")
print(f"\n🔧 Información del sistema:")
print(f"   Python: {sys.version.split()[0]}")
print(f"   PyTorch: {torch.__version__}")

# Detectar aceleración disponible en macOS
if torch.backends.mps.is_available():
    device = 'mps'  # Metal Performance Shaders (Apple Silicon)
    print(f"   🚀 Aceleración: MPS (Metal) - Apple Silicon")
elif torch.cuda.is_available():
    device = 'cuda'
    print(f"   🚀 Aceleración: CUDA")
else:
    device = 'cpu'
    print(f"   💻 Aceleración: CPU")

print(f"   Dispositivo seleccionado: {device}")


✅ Librerías importadas correctamente

🔧 Información del sistema:
   Python: 3.11.13
   PyTorch: 2.9.0
   🚀 Aceleración: MPS (Metal) - Apple Silicon
   Dispositivo seleccionado: mps


In [16]:
# ============================================
# CELDA 4: Cargar modelo
# ============================================
# Modelos disponibles: 'n' (nano), 's' (small), 'm' (medium), 'l' (large), 'x' (xlarge)
# Para macOS, recomiendo 'n' o 's' para mejor rendimiento
MODEL_SIZE = 'n'

print(f"\n🤖 Cargando modelo YOLOv11-{MODEL_SIZE}...")
model = YOLO(f'yolo11{MODEL_SIZE}.pt')

# Configurar dispositivo
if device != 'cpu':
    model.to(device)
    print(f"✅ Modelo cargado en {device.upper()}")
else:
    print(f"✅ Modelo cargado en CPU")


🤖 Cargando modelo YOLOv11-n...
✅ Modelo cargado en MPS


In [18]:
# ============================================
# CELDA 5: Configuración
# ============================================
# ⬇️ CONFIGURA ESTOS PARÁMETROS ⬇️
VIDEO_PATH = '2103099-uhd_3840_2160_30fps.mp4'  # ⬅️ CAMBIA ESTO
OUTPUT_PATH = 'video_con_detecciones.mp4'
CONF_THRESHOLD = 0.4  # Umbral de confianza (0.0 - 1.0)

# Verificar archivo
if Path(VIDEO_PATH).exists():
    print(f"\n✅ Video encontrado: {VIDEO_PATH}")
    # Obtener información del video
    cap_test = cv2.VideoCapture(VIDEO_PATH)
    if cap_test.isOpened():
        width = int(cap_test.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap_test.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap_test.get(cv2.CAP_PROP_FPS))
        frames = int(cap_test.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = frames / fps if fps > 0 else 0
        print(f"   📐 Resolución: {width}x{height}")
        print(f"   🎞️  FPS: {fps}")
        print(f"   ⏱️  Duración: {duration:.2f} segundos")
        cap_test.release()
else:
    print(f"\n❌ Error: No se encontró '{VIDEO_PATH}'")
    print(f"💡 Asegúrate de colocar el video en la misma carpeta que este script")


✅ Video encontrado: 2103099-uhd_3840_2160_30fps.mp4
   📐 Resolución: 3840x2160
   🎞️  FPS: 30
   ⏱️  Duración: 60.00 segundos


In [19]:
# ============================================
# CELDA 6: Función de detección (optimizada para macOS)
# ============================================
def detect_vehicles_macos(video_path, output_path, conf_threshold=0.4, device='cpu'):
    """
    Detecta vehículos en video - Optimizado para macOS
    """
    
    # Verificar codec compatible con macOS
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ No se pudo abrir el video")
        return None
    
    # Propiedades del video
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"\n📊 Información del video:")
    print(f"   Resolución: {width}x{height}")
    print(f"   FPS: {fps}")
    print(f"   Total frames: {total_frames}")
    print(f"   Dispositivo: {device.upper()}")
    
    # Codec para macOS - usar 'avc1' para mejor compatibilidad
    if sys.platform == 'darwin':  # macOS
        fourcc = cv2.VideoWriter_fourcc(*'avc1')  # H.264
    else:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    if not out.isOpened():
        print("⚠️  Intentando con codec alternativo...")
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    # Clases de vehículos (COCO dataset)
    vehicle_classes = {
        2: 'Auto',
        3: 'Moto', 
        5: 'Bus',
        7: 'Camión'
    }
    
    colors = {
        2: (0, 255, 0),    # Verde
        3: (255, 0, 0),    # Azul
        5: (0, 255, 255),  # Amarillo
        7: (255, 0, 255)   # Magenta
    }
    
    print(f"\n🎬 Procesando video...\n")
    
    frame_count = 0
    max_vehicles = 0
    stats = {cls: 0 for cls in vehicle_classes.keys()}
    
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_count += 1
            
            # Detección con el dispositivo especificado
            results = model(frame, conf=conf_threshold, verbose=False, device=device)
            
            vehicle_counts = {cls: 0 for cls in vehicle_classes.keys()}
            
            # Procesar detecciones
            for result in results:
                boxes = result.boxes
                for box in boxes:
                    cls = int(box.cls[0])
                    
                    if cls in vehicle_classes:
                        vehicle_counts[cls] += 1
                        stats[cls] = max(stats[cls], vehicle_counts[cls])
                        
                        # Coordenadas
                        x1, y1, x2, y2 = map(int, box.xyxy[0])
                        conf = float(box.conf[0])
                        
                        # Dibujar rectángulo
                        color = colors[cls]
                        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                        
                        # Etiqueta con fondo
                        label = f'{vehicle_classes[cls]} {conf:.2f}'
                        label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)[0]
                        cv2.rectangle(frame, (x1, y1 - label_size[1] - 10), 
                                    (x1 + label_size[0], y1), color, -1)
                        cv2.putText(frame, label, (x1, y1 - 5),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
            
            # Panel de estadísticas
            total_vehicles = sum(vehicle_counts.values())
            max_vehicles = max(max_vehicles, total_vehicles)
            
            active_classes = [cls for cls in vehicle_classes.keys() if vehicle_counts[cls] > 0]
            panel_height = 40 + len(active_classes) * 25
            
            cv2.rectangle(frame, (10, 10), (260, panel_height), (0, 0, 0), -1)
            cv2.rectangle(frame, (10, 10), (260, panel_height), (255, 255, 255), 2)
            
            y_pos = 35
            cv2.putText(frame, f'Total Vehiculos: {total_vehicles}', (20, y_pos),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            
            y_pos += 30
            for cls in active_classes:
                name = vehicle_classes[cls]
                cv2.putText(frame, f'{name}: {vehicle_counts[cls]}', 
                          (20, y_pos), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 
                          colors[cls], 2)
                y_pos += 25
            
            # Escribir frame
            out.write(frame)
            
            # Progreso cada 30 frames
            if frame_count % 30 == 0:
                progress = (frame_count / total_frames) * 100
                print(f"⏳ {progress:.1f}% [{frame_count}/{total_frames}] - Vehículos: {total_vehicles}")
        
        print(f"\n✅ ¡Procesamiento completado!")
        print(f"\n📊 Estadísticas finales:")
        print(f"   Frames procesados: {frame_count}")
        print(f"   Máximo vehículos simultáneos: {max_vehicles}")
        for cls, name in vehicle_classes.items():
            if stats[cls] > 0:
                print(f"   Máximo de {name}s: {stats[cls]}")
        print(f"\n💾 Video guardado: {output_path}")
        
        return output_path
        
    except KeyboardInterrupt:
        print("\n⚠️ Procesamiento interrumpido por el usuario")
        return None
    
    except Exception as e:
        print(f"\n❌ Error: {e}")
        return None
    
    finally:
        cap.release()
        out.release()
        cv2.destroyAllWindows()

print("✅ Función de detección lista")

✅ Función de detección lista


In [20]:
# ============================================
# CELDA 7: Procesar video
# ============================================
if Path(VIDEO_PATH).exists():
    print(f"\n🚀 Iniciando detección de vehículos...")
    output_video = detect_vehicles_macos(
        video_path=VIDEO_PATH,
        output_path=OUTPUT_PATH,
        conf_threshold=CONF_THRESHOLD,
        device=device
    )
    
    if output_video:
        print(f"\n🎉 ¡Todo listo!")
        print(f"📹 Abre tu video: {output_video}")
        
        # Abrir automáticamente en macOS
        if sys.platform == 'darwin':
            print(f"\n💡 Abriendo video en QuickTime...")
            os.system(f'open "{output_video}"')
else:
    print(f"\n⚠️ Por favor, coloca tu video y actualiza VIDEO_PATH")


🚀 Iniciando detección de vehículos...

📊 Información del video:
   Resolución: 3840x2160
   FPS: 30
   Total frames: 1800
   Dispositivo: MPS

🎬 Procesando video...

⏳ 1.7% [30/1800] - Vehículos: 14
⏳ 3.3% [60/1800] - Vehículos: 13
⏳ 5.0% [90/1800] - Vehículos: 12
⏳ 6.7% [120/1800] - Vehículos: 14
⏳ 8.3% [150/1800] - Vehículos: 14
⏳ 10.0% [180/1800] - Vehículos: 13
⏳ 11.7% [210/1800] - Vehículos: 12
⏳ 13.3% [240/1800] - Vehículos: 10
⏳ 15.0% [270/1800] - Vehículos: 16
⏳ 16.7% [300/1800] - Vehículos: 14
⏳ 18.3% [330/1800] - Vehículos: 15
⏳ 20.0% [360/1800] - Vehículos: 14
⏳ 21.7% [390/1800] - Vehículos: 15
⏳ 23.3% [420/1800] - Vehículos: 15
⏳ 25.0% [450/1800] - Vehículos: 15
⏳ 26.7% [480/1800] - Vehículos: 14
⏳ 28.3% [510/1800] - Vehículos: 13
⏳ 30.0% [540/1800] - Vehículos: 14
⏳ 31.7% [570/1800] - Vehículos: 14
⏳ 33.3% [600/1800] - Vehículos: 17
⏳ 35.0% [630/1800] - Vehículos: 16
⏳ 36.7% [660/1800] - Vehículos: 12
⏳ 38.3% [690/1800] - Vehículos: 12
⏳ 40.0% [720/1800] - Vehículos: 13
⏳